In [1]:
import cv2
import numpy as np
import glob, os

In [3]:
# Check the percent of hero pictures 
data_dir = "../data/train"
mask_dir = "../data/train/masks"
img_dir = "../data/train/images"
mask_path = os.path.join(mask_dir, "*.png")
img_path = os.path.join(img_dir, "*.png")

In [2]:
total_files = 0
total_hero = 0
total_num = 0

for file in glob.glob(mask_path):
    total_files +=1
    img = cv2.imread(file)
    blue = img[:,:,0]
    green = img[:,0,:]

    if np.any(blue == 255):
        total_hero += 1
    if np.any(blue == 255) or np.any(green == 255):
        total_num += 1

percent_hero = 100. * total_hero / total_files
percent_num = 100. * total_num / total_files

print (percent_hero, "percent of files contain the hero")
print(percent_num, "percent of files contain people")

44.49213161659514 percent of files contain the hero
100.0 percent of files contain people


In [24]:
mask_path

'../data/train/masks/*.png'

In [11]:
# data cleaner masks images


# 0_run2_cam1_00000.jpeg
# 0_run2_mask_00000.png
total_num = 0
file_num = 0
for file in glob.glob(mask_path):
    total_num += 1
    img = cv2.imread(file)
    red = img[0,:,:]
    blue = img[:,:,0]
    green = img[:,0,:]
    
    if not (np.any(blue>50) or np.any(green>50)):
        file_num += 1
        
print(file_num, total_num)


0 4893
